In [ ]:
import cupy as cp
import numpy as np
import time
import math
import matplotlib.pyplot as plt
import sys
sys.path.append('/software/pc24403/tfpcbpggsz/func')
sys.path.append('/software/pc24403/tfpcbpggsz/amp')
from PyD0ToKSpipi2018 import *
import uproot as up
import pickle
import os
from multiprocessing import Pool

In [ ]:
def get_mass(p1,p2):
    return ((p1[:,0]+p2[:,0])**2 - (p1[:,1]+p2[:,1])**2 - (p1[:,2]+p2[:,2])**2 - (p1[:,3]+p2[:,3])**2)

def get_p4(decay="b2dpi", cut='', index=1):

    file_name = ''
    if cut == 'int':
        file_name = f'/software/pc24403/PCBPGGSZ/Int/weighted_{decay}.root:DalitzEventList'
    
    elif decay.split('_')[0] == 'b2dk' or decay.split('_')[0] == 'b2dpi':
        if cut == 'p':
            file_name = f'/software/pc24403/PCBPGGSZ/outputs/toy/mass_fit/add_sw/new_frac_sw_pg/lhcb_toy_{decay}_{index}_CPrange.root:BplusDalitzEventList'
        elif cut == 'm':
            file_name = f'/software/pc24403/PCBPGGSZ/outputs/oy/mass_fit/add_sw/new_frac_sw_pg/lhcb_toy_{decay}_{index}.root:BminusDalitzEventList'

    tree = up.open(file_name)
  # Load the branches as arrays
    branch_names = ["_1_K0S0_E", "_1_K0S0_Px", "_1_K0S0_Py", "_1_K0S0_Pz",
                         "_2_pi#_E", "_2_pi#_Px", "_2_pi#_Py", "_2_pi#_Pz",
                         "_3_pi~_E", "_3_pi~_Px", "_3_pi~_Py", "_3_pi~_Pz"]
    array = tree.arrays(branch_names)
       

    p1 = cp.asarray([array["_1_K0S0_E"], array["_1_K0S0_Px"], array["_1_K0S0_Py"], array["_1_K0S0_Pz"]])
    p2 = cp.asarray([array["_2_pi#_E"], array["_2_pi#_Px"], array["_2_pi#_Py"], array["_2_pi#_Pz"]])
    p3 = cp.asarray([array["_3_pi~_E"], array["_3_pi~_Px"], array["_3_pi~_Py"], array["_3_pi~_Pz"]])
    
    # convert 4*1000 into a vectot<double>
    p1 = cp.transpose(p1)
    p2 = cp.transpose(p2)
    p3 = cp.transpose(p3)


    return p1, p2, p3

def fillamplitude(amp, events):
    